In [42]:
import torch
from torch import nn, optim
import numpy as np

def create_model(num_features, sequence_len, channels, k):
    network = nn.Sequential()
    
    # first convolutional layer, followed by relu and maxpooling layers
    network.add_module('Conv1', nn.Conv2d(in_channels=channels, out_channels=4*channels, kernel_size=(1, num_features)))
    network.add_module('Conv2', nn.Conv2d(in_channels=4*channels, out_channels=8*channels, kernel_size=(k, 1)))
    network.add_module('ReLU1', nn.ReLU())
    network.add_module('MaxPool1', nn.MaxPool2d(kernel_size=(2,1), stride=2))
    
    # next layer, same thing.
    network.add_module('Conv3', nn.Conv2d(in_channels=8*channels, out_channels=16*channels, kernel_size=(k, 1)))
    network.add_module('ReLU2', nn.ReLU())
    network.add_module('MaxPool2', nn.MaxPool2d(kernel_size=(2,1), stride=2))
    
    # then flatten everything into a single dimension and apply dropout
    network.add_module('Flatten', nn.Flatten())
    network.add_module('Dropout', nn.Dropout(0.2))
    
    # fully connected layers to produce a new set of features
    flat_size = 16 * channels * ((sequence_len - k + 1) // 2 - k + 1) // 2
    network.add_module('FC1', nn.Linear(flat_size, 512))
    network.add_module('ReLU3', nn.ReLU())
    network.add_module('FC2', nn.Linear(512, 128))
    network.add_module('ReLU4', nn.ReLU())
    network.add_module('FC3', nn.Linear(128, num_features))

    return network

In [43]:
historical_data = r"data\grand_island_nwt.csv"
normals_data = r"data\grand_island_normals.csv"

import pandas as pd

historical = pd.read_csv(historical_data)
normals = pd.read_csv(normals_data)
normals.insert(0, 'DAY', normals.index)


In [44]:
import torch

def create_tensors(historical_data: pd.DataFrame, normal_data: pd.DataFrame):
    date_lookup = {}
    index = historical_data['DATE']
    for i in range(len(index)):
        date_lookup[index[i]] = i

    merged = pd.merge(historical_data, normal_data, on='DAY', how='left')
    mean_columns = [x for x in merged.columns if 'mean' in x]
    normals_for_historical = merged[['DATE', 'DAY'] + mean_columns]
    normals_for_historical.columns = historical_data.columns
    differences = historical_data.drop(['DATE', 'DAY'], axis=1).subtract(normals_for_historical.drop(['DATE', 'DAY'], axis=1))
    squared_differences = differences.pow(2)

    historical = historical_data.drop(['DATE', 'DAY'], axis=1)
    normal = normals_for_historical.drop(['DATE', 'DAY'], axis=1)

    h_tensors = torch.from_numpy(historical.values)
    n_tensors = torch.from_numpy(normal.values)
    d_tensors = torch.from_numpy(differences.values)
    s_tensors = torch.from_numpy(squared_differences.values)

    return torch.stack((h_tensors, n_tensors, d_tensors, s_tensors), dim=0), date_lookup

t, dl = create_tensors(historical, normals)


In [50]:
def get_historical_features(frm, tensors, date_lookup, input=False):
    if isinstance(frm, str):
        from_idx = date_lookup[frm]
    else:
        from_idx = frm
    if input:
        return tensors[:, from_idx, :]
    else:
        return tensors[0, from_idx, :]

def get_many_historical_features(frm, to, tensors, date_lookup, input=False):
    if isinstance(frm, str):
        from_idx = date_lookup[frm]
    else:
        from_idx = frm

    if isinstance(to, str):
        to_idx = date_lookup[to]
    else:
        to_idx = to

    data = []
    for idx in range(from_idx, to_idx + 1, 1):
        if input:
            data.append(tensors[:, idx, :])
        else:
            data.append(tensors[0, idx, :])

    if input:
        return torch.stack(data, dim=1)
    else:
        return torch.stack(data, dim=0)

def create_features_datasets(tensors, date_lookup, backward_features, forward_features):
    first_idx = backward_features + 1
    last_idx = tensors.size()[1] - forward_features - 1
    
    X = []
    Y = []

    for idx in range(first_idx, last_idx, 1):
        if backward_features > 0 and forward_features > 0:
            backward = get_many_historical_features(idx - backward_features, idx - 1, tensors, date_lookup, input=True)
            forward = get_many_historical_features(idx + 1, idx + forward_features, tensors, date_lookup, input=True)            
            input_features = torch.concat((backward, forward), dim=1)
        elif backward_features > 0:
            input_features = get_many_historical_features(idx - backward_features, idx - 1, tensors, date_lookup, input=True)
        elif forward_features > 0:
            input_features = get_many_historical_features(idx + 1, idx + forward_features, tensors, date_lookup, input=True)            

        output_features = get_historical_features(idx, tensors, date_lookup, input=False)

        X.append(input_features)
        Y.append(output_features)
    
    return torch.stack(X, dim=0), torch.stack(Y, dim=0)

In [51]:
import torch.utils.data as Data

X, Y = create_features_datasets(t, dl, 20, 0)

print(X.size())
print(Y.size())

dataset = Data.TensorDataset(X, Y)
loader = Data.DataLoader(dataset, batch_size=8, shuffle=True)

for X_batch, Y_batch in loader:
    print(X_batch.size())
    print(Y_batch.size())
    break

torch.Size([14954, 4, 20, 13])
torch.Size([14954, 13])
torch.Size([8, 4, 20, 13])
torch.Size([8, 13])


In [52]:
model = create_model(13, 20, 4, 3)

model(X_batch.float())

RuntimeError: mat1 and mat2 shapes cannot be multiplied (8x192 and 224x512)